In [1]:
from __future__ import absolute_import, print_function
import pandas as pd
import boto3
from botocore.client import Config
from flytekit.common import utils as _utils
from flytekit.sdk import test_utils as _test_utils
from flytekit.configuration import set_flyte_config_file
from flytekit.common.tasks.presto_task import SdkPrestoTask
from flytekit.sdk.tasks import inputs
from flytekit.sdk.types import Types

set_flyte_config_file("/Users/hamabuelfutuh/.flyte/config")

def print_console_url(exc):
    print("http://flyte.lyft.net/console/projects/{}/domains/{}/executions/{}".format(exc.id.project, exc.id.domain, exc.id.name)) 

s3 = boto3.resource('s3',
                    endpoint_url='http://localhost:30084',
                    aws_access_key_id='minio',
                    aws_secret_access_key='miniostorage',
                    config=Config(signature_version='s3v4'),
                    region_name='us-east-1')

def upload_file(f, ref):
    mod = ref.lstrip("s3://")
    bucket, path = mod.split("/", 1)
    s3.Bucket(bucket).upload_file('image.py',path)
    
from IPython.display import Image, display
def display_images(paths):
    for p in paths:
        display(Image(p))

def print_schema(schema):
    with _test_utils.LocalTestFileSystem() as sandbox:
        # load schema data
        schema.download()
        df = pd.read_parquet(schema.local_path)
    print(df)

In [4]:
get_data_task = SdkPrestoTask(
    task_inputs=inputs(ds=Types.String),
    statement="""
    SELECT * 
    FROM hive.flyte.datacouncildemo_train
    """,
    output_schema=Types.Schema(),
    discoverable=True,
    discovery_version="2",
)

task_exec = get_data_task.register_and_launch(project="flytekit", domain="development", inputs={"ds": '2020-07-05'})
print("Created execution.")
print_console_url(task_exec)
print("Waiting for execution to complete...")
task_exec.wait_for_completion()
print("Done!")

Created execution.
http://flyte.lyft.net/console/projects/flytekit/domains/development/executions/zkrlfeonib
Waiting for execution to complete...
Done!


In [5]:
print_schema(task_exec.outputs['results'])

    col0 col1 col2 col3 col4  col5   col6 col7 col8
0      8  183   64    0    0  23.3  0.672   32    1
1      1   89   66   23   94  28.1  0.167   21    0
2      0  137   40   35  168  43.1  2.288   33    1
3      5  116   74    0    0  25.6  0.201   30    0
4      3   78   50   32   88  31.0  0.248   26    1
..   ...  ...  ...  ...  ...   ...    ...  ...  ...
762   10  101   76   48  180  32.9  0.171   63    0
763    2  122   70   27    0  36.8  0.340   27    0
764    5  121   72   23  112  26.2  0.245   30    0
765    1  126   60    0    0  30.1  0.349   47    1
766    1   93   70   31    0  30.4  0.315   23    0

[767 rows x 9 columns]
